In [1]:
import xarray as xr
import numpy as np

from dask.distributed import Client

In [2]:
client = Client()
client

Client Scheduler: tcp://127.0.0.1:58764 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 8.50 GB


In [3]:
ds = xr.Dataset()
coords = {"time": np.arange(0, 400, 1), "y":np.arange(30_000.0, 0.0, -100.0) - 50.0, "x": np.arange(0.0, 20_000.0, 100.0) + 50.0}
dims = ("time", "y", "x")
ds["primary"] = xr.DataArray(np.random.rand(400, 300, 200), coords, dims)
ds["secondary"] = xr.DataArray(np.random.rand(400, 300, 200), coords, dims)
ds["tertiary"] = xr.DataArray(np.random.rand(400, 300, 200), coords, dims)
ds.to_netcdf("flux.nc")

### "Sensible" chunking

In [4]:
ds = xr.open_dataset("flux.nc", chunks={"time":1})
ds["total"] = ds["primary"] + ds["secondary"] + ds["tertiary"]

In [5]:
%%time
ds.to_netcdf("flux_with_total.nc")

Wall time: 1min 1s


In [6]:
ds.close()

### Suboptimal chunking

In [7]:
ds = xr.open_dataset("flux.nc", chunks={"y":50, "x":50})
ds["total"] = ds["primary"] + ds["secondary"] + ds["tertiary"]

In [8]:
%%time
ds.to_netcdf("flux_with_total.nc")

Wall time: 12.5 s


In [9]:
ds.close()